In [ ]:
!pip install transformers
!pip install datasets
!pip install numpy
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
def tokenize(text):
    prompts = text['Prompts']
    tokenized_prompts = tokenizer(prompts, padding="max_length", truncation=True, max_length=256)
    labels = np.zeros(len(prompts))  # Create an array of zeros with the same length as 'Prompts'
    return {'input_ids': np.array(tokenized_prompts['input_ids']), 'labels': labels}


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


# Load the dataset
dataset = load_dataset("vedantgaur/GPTOutputs-MWP", trust_remote_code=True)

# Add a new field 'labels' that always contains the value 0 (representing 'human generated')
dataset = dataset.map(lambda example: {'labels': 0})

# Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizedDataset = dataset.map(tokenize, batched=True)

# Split into train and test sets
trainDataset = tokenizedDataset['train']
testDataset = tokenizedDataset['validation']

# Model: bert-base-uncased
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

BertTraining_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=3,
)

# Creation of Bert Trainer Object
trainer = Trainer(
    model=model,
    args=BertTraining_args,
    train_dataset=trainDataset,
    eval_dataset=testDataset,
    compute_metrics=compute_metrics,
)

# Fine-tuning the Model
trainer.train()



Model: bert-base-uncased


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,0.001900


TrainOutput(global_step=573, training_loss=0.0016259829126490219, metrics={'train_runtime': 413.3457, 'train_samples_per_second': 22.122, 'train_steps_per_second': 1.386, 'total_flos': 1202943745105920.0, 'train_loss': 0.0016259829126490219, 'epoch': 3.0})

In [ ]:
# Predictions
metric = load_metric("accuracy")
predictions = trainer.predict(testDataset)
classPredictions = np.argmax(predictions.predictions, axis=-1)
evalMetrics = compute_metrics((predictions.predictions, predictions.label_ids))
print(evalMetrics)

apiToken = "REDACTED"

# Push the fine-tuned model to Huggingface
model.push_to_hub("SkwarczynskiP/bert-base-uncased-finetuned-vedantgaur-human-generated", token=apiToken)

# Push the tokenizer to Huggingface
tokenizer.push_to_hub("SkwarczynskiP/bert-base-uncased-finetuned-vedantgaur-human-generated", token=apiToken)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'accuracy': 1.0}


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SkwarczynskiP/bert-base-uncased-finetuned-vedantgaur-human-generated/commit/f946c59fdd41314e770eb6f8093a32ace7134908', commit_message='Upload tokenizer', commit_description='', oid='f946c59fdd41314e770eb6f8093a32ace7134908', pr_url=None, pr_revision=None, pr_num=None)